# Two-Pointer Pattern Guide (Interview Prep, Deep Explanation)

This notebook mirrors `two_pointers_guide.md` and `two_pointers_examples.py` with the same APIs and sample validations.

## 1) Two-Pointer Concept, Trigger Signals, and Variants

Two pointers means maintaining two indices that move with strict rules to avoid repeated work.

Trigger signals:
1. Pair/triple relationships in arrays/strings.
2. Sorted data (or sortable data).
3. Symmetry checks (palindrome style).
4. Need for low extra memory.

Variants:
1. Opposite ends moving inward.
2. Same direction fast/slow pointers.
3. Fixed index + left/right scan (3Sum).

## 2) Why Brute Force First

Brute force defines the correctness target and exposes where time is wasted.
Two pointers then remove repeated search by monotonic pointer movement.

## 3) Shared Explanation Template

For each problem we include:
1. Problem statement
2. Brute-force idea and complexity
3. Two-pointer optimization
4. Loop invariant / correctness reasoning
5. Dry run
6. Complexity comparison

## 4) Problem 1: Valid Palindrome (LeetCode 125)

### Statement
Return `True` if string `s` is palindrome after filtering non-alphanumeric chars and ignoring case.

### Brute Force
- Build cleaned lowercase string and compare with reverse.
- Time `O(n)`, space `O(n)`.

### Two Pointer
- `left` from start, `right` from end.
- Skip irrelevant chars, compare lowercase values, move inward.

### Invariant
Before each comparison, all checked mirrored pairs already match; `left/right` identify the next unchecked candidates.

### Dry Run (A man, a plan, a canal: Panama)
- Compare `A` vs `a` -> match
- Skip punctuation/spaces
- Continue until pointers cross -> `True`.

## 5) Problem 2: 3Sum (LeetCode 15)

### Statement
Find all unique triplets `[a,b,c]` such that `a+b+c == 0`.

### Output Normalization Rule
1. Each triplet sorted ascending.
2. Full result sorted lexicographically.
3. No duplicates.

### Brute Force
- Triple nested loops over `i<j<k`.
- Use set of sorted tuples for dedup.
- Time `O(n^3)`.

### Two Pointer
- Sort array.
- Fix anchor `i`; scan suffix with `left/right`.
- Move `left` if sum small, move `right` if sum large, record if zero.
- Skip duplicates for anchor and moving pointers.

### Invariant
For fixed `i`, unprocessed pair candidates are exactly in `[left, right]`; sorted monotonicity justifies one-direction moves.

### Dry Run
`[-1,0,1,2,-1,-4]` -> sorted `[-4,-1,-1,0,1,2]`
- `i=-4`: all sums too small.
- `i=-1`: find `[-1,-1,2]` and `[-1,0,1]`.
- Skip duplicate anchor `-1`.

In [ ]:
from __future__ import annotations

def is_palindrome_bruteforce(s: str) -> bool:
    cleaned = "".join(ch.lower() for ch in s if ch.isalnum())
    return cleaned == cleaned[::-1]


def is_palindrome_two_pointer(s: str) -> bool:
    left = 0
    right = len(s) - 1

    while left < right:
        while left < right and not s[left].isalnum():
            left += 1
        while left < right and not s[right].isalnum():
            right -= 1

        if s[left].lower() != s[right].lower():
            return False

        left += 1
        right -= 1

    return True


def _normalize_triplets(triplets: set[tuple[int, int, int]] | list[list[int]]) -> list[list[int]]:
    normalized_set: set[tuple[int, int, int]] = set()

    if isinstance(triplets, set):
        for triplet in triplets:
            normalized_set.add(tuple(sorted(triplet)))
    else:
        for triplet in triplets:
            normalized_set.add(tuple(sorted(triplet)))

    return [list(t) for t in sorted(normalized_set)]


def three_sum_bruteforce(nums: list[int]) -> list[list[int]]:
    n = len(nums)
    results: set[tuple[int, int, int]] = set()

    for i in range(n):
        for j in range(i + 1, n):
            for k in range(j + 1, n):
                if nums[i] + nums[j] + nums[k] == 0:
                    results.add(tuple(sorted((nums[i], nums[j], nums[k]))))

    return _normalize_triplets(results)


def three_sum_two_pointer(nums: list[int]) -> list[list[int]]:
    arr = sorted(nums)
    n = len(arr)
    results: list[list[int]] = []

    for i in range(n - 2):
        if i > 0 and arr[i] == arr[i - 1]:
            continue

        left = i + 1
        right = n - 1

        while left < right:
            total = arr[i] + arr[left] + arr[right]

            if total == 0:
                results.append([arr[i], arr[left], arr[right]])
                left += 1
                right -= 1

                while left < right and arr[left] == arr[left - 1]:
                    left += 1
                while left < right and arr[right] == arr[right + 1]:
                    right -= 1
            elif total < 0:
                left += 1
            else:
                right -= 1

    return _normalize_triplets(results)


def _print_test_result(name: str, passed: bool) -> None:
    status = "PASS" if passed else "FAIL"
    print(f"[{status}] {name}")


def run_demo() -> None:
    palindrome_cases: list[tuple[str, bool]] = [
        ("A man, a plan, a canal: Panama", True),
        ("race a car", False),
        ("", True),
        (".,", True),
        ("0P", False),
    ]

    print("Palindrome Tests")
    for text, expected in palindrome_cases:
        brute = is_palindrome_bruteforce(text)
        optimized = is_palindrome_two_pointer(text)
        _print_test_result(
            f"input={text!r} expected={expected} brute={brute} optimized={optimized}",
            brute == optimized and optimized == expected,
        )

    print()

    three_sum_cases: list[tuple[list[int], list[list[int]]]] = [
        ([-1, 0, 1, 2, -1, -4], [[-1, -1, 2], [-1, 0, 1]]),
        ([0, 0, 0, 0], [[0, 0, 0]]),
        ([1, 2, -2, -1], []),
        ([-2, 0, 1, 1, 2], [[-2, 0, 2], [-2, 1, 1]]),
    ]

    print("3Sum Tests")
    for nums, expected in three_sum_cases:
        brute = three_sum_bruteforce(nums)
        optimized = three_sum_two_pointer(nums)
        dedup_ok = len(optimized) == len({tuple(t) for t in optimized})
        _print_test_result(
            f"input={nums} expected={expected} brute={brute} optimized={optimized} dedup_ok={dedup_ok}",
            dedup_ok and brute == optimized and optimized == expected,
        )

    print()
    print("Validation checks completed.")

In [ ]:
run_demo()

## 6) Pattern Comparison

| Problem | Brute Force | Two Pointer | Main Gain |
|---|---|---|---|
| Valid Palindrome | clean + reverse | inward scan with skips | lower extra space |
| 3Sum | triple loop | sort + anchor + pair scan | time reduced from O(n^3) to O(n^2) |

## 7) Interview Checklist

1. Explain why two pointers applies.
2. State exact pointer move rule.
3. Handle duplicates explicitly (3Sum).
4. Mention invariant and termination.
5. Give exact complexity tradeoff.